In [1]:
#%%capture
%matplotlib inline
import seaborn as sns
sns.set_context('notebook')
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
plt.plot([0,1],[1,0])
%matplotlib notebook
import matplotlib.pyplot as plt
#pip install natsort

#pip install igor
"""NeuronUnit module for interaction with the Blue Brain Project data."""
import os
import zipfile
import json
import IPython
import requests
import matplotlib.pyplot as plt
from neo.io import IgorIO
import natsort
from neuronunit.capabilities import spike_functions
import quantities as pq
import numpy as np
try:  # Python 3
    from io import BytesIO
    from urllib.request import urlopen, URLError
    MAJOR_VERSION = 3
except ImportError:  # Python 2
    from StringIO import StringIO
    from urllib2 import urlopen, URLError
    MAJOR_VERSION = 2


def is_bbp_up():
    """Check whether the BBP microcircuit portal is up."""
    url = "http://microcircuits.epfl.ch/released_data/B95_folder.zip"
    request = requests.get(url)
    return request.status_code == 200


def list_curated_data():
    """List all curated datasets as of July 1st, 2017.
    Includes those found at
    http://microcircuits.epfl.ch/#/article/article_4_eph
    """
    url = "http://microcircuits.epfl.ch/data/articles/article_4_eph.json"
    cells = []
    try:
        response = urlopen(url)
    except URLError:
        print ("Could not find list of curated data at %s" % URL)
    else:
        data = json.load(response)
        table = data['data_table']['table']['rows']
        for section in table:
            for row in section:
                if 'term' in row:
                    cell = row['term'].split(' ')[1]
                    cells.append(cell)
    return cells


def get_curated_data(data_id, sweeps=None):
    """Download curated data (Igor files) from the microcircuit portal.
    data_id: An ID number like the ones in 'list_curated_data()' that appears
    in http://microcircuits.epfl.ch/#/article/article_4_eph.
    """
    url = "http://microcircuits.epfl.ch/data/released_data/%s.zip" % data_id
    data = get_sweeps(url, sweeps=sweeps)
    return data


def get_uncurated_data(data_id, sweeps=None):
    """Download uncurated data (Igor files) from the microcircuit portal."""
    url = "http://microcircuits.epfl.ch/data/uncurated/%s_folder.zip" % data_id
    data = get_sweeps(url, sweeps=sweeps)
    return data


def get_sweeps(url, sweeps=None):
    """Get sweeps of data from the given URL."""
    print("Getting data from %s" % url)
    path = find_or_download_data(url)  # Base path for this data
    assert type(sweeps) in [type(None), list], "Sweeps must be None or a list."
    sweep_paths = list_sweeps(path)  # Available sweeps
    if sweeps is None:
        sweeps = sweep_paths
    else:
        sweeps = []
        for sweep_path in sweep_paths:
            if any([sweep_path.endswith(sweep for sweep in sweeps)]):
                sweeps.append(sweep_path)
        sweeps = set(sweeps)
    data = {sweep: open_data(sweep) for sweep in sweeps}
    return data


def find_or_download_data(url):
    """Find or download data from the given URL.
    Return a path to a local directory containing the unzipped data found
    at the provided url.  The zipped file will be downloaded and unzipped if
    the directory cannot be found.  The path to the directory is returned.
    """
    zipped = url.split('/')[-1]  # Name of zip file
    unzipped = zipped.split('.')[0]  # Name when unzipped
    z = None
    if not os.path.isdir(unzipped):  # If unzipped version not found
        if MAJOR_VERSION == 2:
            r = requests.get(url, stream=True)
            z = zipfile.ZipFile(StringIO(r.content))
        elif MAJOR_VERSION == 3:
            r = requests.get(url)
            z = zipfile.ZipFile(BytesIO(r.content))
        z.extractall(unzipped)
    return unzipped


def list_sweeps(url, extension='.ibw'):
    """List all sweeps available in the file at the given URL."""
    path = find_or_download_data(url)  # Base path for this data
    sweeps = find_sweeps(path, extension=extension)
    return sweeps


def find_sweeps(path, extension='.ibw', depth=0):
    """Find sweeps available at the given path.
    Starting from 'path', recursively searches subdirectories and returns
    full paths to all files ending with 'extension'.
    """
    sweeps = []
    items = os.listdir(path)
    for item in items:
        new_path = os.path.join(path, item)
        if os.path.isdir(new_path):
            sweeps += find_sweeps(new_path, extension=extension, depth=depth+1)
        if os.path.isfile(new_path) and item.endswith(extension):
            sweeps += [new_path]
    return sweeps


def open_data(path):
    """Take a 'path' to an .ibw file and returns a neo.core.AnalogSignal."""
    igor_io = IgorIO(filename=path)
    analog_signal = igor_io.read_analogsignal()
    return analog_signal


def plot_data(signal,current):
    """Plot the data in a neo.core.AnalogSignal."""
    plt.clf()
    plt.plot(signal.times, signal)
    plt.xlabel(signal.sampling_period.dimensionality)
    plt.ylabel(signal.dimensionality)
    plt.title(np.max(current))
    plt.show()
    
    


In [2]:
import pickle
import pandas as pd
models_hbp = pickle.load(open("hbp_data2.p","rb"))
bbp = pd.DataFrame([m.everything for m in models_hbp if m is not None and hasattr(m,'everything')])
bbp

,AHP1_depth_from_peak,AHP2_depth_from_peak,AHP_depth,AHP_depth_abs,AHP_depth_abs_slow,AHP_depth_diff,AHP_depth_from_peak,AHP_slow_time,AHP_time_from_peak,AP1_amp,...,ISICVTest,ISIMedianTest,ISIBurstMeanChangeTest,SpikeRateStrongStimTest,AP1DelayMeanStrongStimTest,AP1DelaySDStrongStimTest,AP2DelayMeanStrongStimTest,AP2DelaySDStrongStimTest,Burst1ISIMeanStrongStimTest,Burst1ISISDStrongStimTest
0,72.227725,70.996537,14.057036,-55.136892,-56.578204,0.132331,69.649465,0.294414,12.241667,66.059276,...,2.403056,207.5 ms,8.108108,3.5 Hz,115.0 ms,0.0 ms,281.5 ms,0.0 ms,173.25 ms,0.0 ms
1,64.340610,58.715882,16.368972,-56.224101,-55.258187,-0.076221,55.444148,0.377729,3.616779,82.608464,...,14.874261,35.25 ms,-4.477612,24.0 Hz,26.75 ms,0.0 ms,60.25 ms,0.0 ms,32.75 ms,0.0 ms
2,96.901524,82.570967,25.283432,-47.933145,-48.451515,0.341650,76.275914,0.218377,7.854839,85.902058,...,NaN,None,NaN,None,None,None,None,None,None,None
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.536992,196.5 ms,150.071124,3.0 Hz,14.250000000000114 ms,0.0 ms,190.0 ms,0.0 ms,307.62499999999994 ms,0.0 ms
4,68.959134,65.615545,23.033218,-47.989932,-47.934178,0.123409,61.399867,0.193024,4.941667,58.809628,...,1.038511,84.375 ms,-80.724234,5.5 Hz,43.0 ms,0.0 ms,491.75 ms,0.0 ms,267.625 ms,0.0 ms
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,77.058742,71.090277,20.711179,-51.207898,-51.052890,0.083769,64.279987,0.282834,4.511818,64.334360,...,4.347594,406.5 ms,-37.400000,1.5 Hz,59.75 ms,0.0 ms,559.75 ms,0.0 ms,406.5 ms,0.0 ms
78,65.528051,47.553930,26.858314,-45.184072,-46.742036,0.044442,54.613503,0.515628,9.060870,66.840487,...,3.621621,463.0 ms,92.027491,2.0 Hz,45.0 ms,0.0 ms,408.75 ms,0.0 ms,531.125 ms,0.0 ms
79,73.502661,57.590940,24.534994,-46.634014,-47.774608,0.075621,64.815385,0.336413,14.490323,73.358921,...,16.330357,457.25 ms,13.046010,1.5 Hz,358.25 ms,0.0 ms,787.5 ms,0.0 ms,457.25 ms,0.0 ms
80,65.228066,54.341097,30.991459,-41.073363,-41.623656,0.136132,49.443542,0.383356,6.696078,60.422052,...,13.990588,89.625 ms,-9.020619,9.5 Hz,24.25 ms,0.0 ms,121.25 ms,0.0 ms,92.625 ms,0.0 ms


In [ ]:
from neuronunit.optimisation.get_three_feature_sets_from_nml_db import three_feature_sets_on_static_models
import pickle
models_hbp = pickle.load(open("hbp_data.p","rb"))
for i,m in enumerate(models_hbp):
    if hasattr(m,'vm30'):
        m.features = three_feature_sets_on_static_models(m,bbp=True)
        everything = {}
        if m.features['efel_15'] is not None and len(m.features['allen_30']) and m.features['efel_30'] is not None:
            everything.update(m.features['efel_15'][0])
            everything.update(m.features['efel_30'][0])
            everything.update(m.features['allen_30'])
        everything.update(m.features['allen_15'])
        everything.update(m.features['dm'])

        m.everything = everything
        pickle.dump(models_hbp,open("hbp_data3.p","wb"))
print(len(models_hbp))

import pickle
import pandas as pd
models_hbp = pickle.load(open("hbp_data3.p","rb"))
bbp = pd.DataFrame([m.everything for m in models_hbp if m is not None and hasattr(m,'everything')])
bbp

INFO:root:test
INFO:numexpr.utils:NumExpr defaulting to 4 threads.


  20 +------------------------------------------------------------------------------------------+
     |                                                                                          |
     |                                                     V_{m} (mV), versus time (ms) ******* |
     |                       *   *                                                              |
     |                     * * * * * *  * *    *  * * *  *  * *  *  * *  *   *  *  *  *         |
     |                     * * * * * *  * *  * *  * * *  *  * *  *  * *  *   *  *  *  *         |
     |                     * * * * * *  * *  * *  * * *  *  * *  *  * *  *   *  *  *  *         |
     |                     * * * * * *  * *  * *  * * *  *  * *  *  * *  *   *  *  *  *         |
  10 |                     * * * * * *  * *  * *  * * *  *  * *  *  * *  *   *  *  *  *         |
     |                     * * * * * *  * *  * *  * * *  *  * *  *  * *  *   *  *  *  *         |
     |              

12000
119990
94
-0.6562957763671875 threshold 5.999707 -76.43377
149
-3.933785915374756 threshold 4.2497926 -79.96484
  60 +------------------------------------------------------------------------------------------+
     |                                                                                          |
     |                                                     V_{m} (mV), versus time (ms) ******* |
     |                                                                                          |
     |                                                                                          |
     |                                                                                          |
     |                                                                                          |
     |                                                                                          |
     |                                                                                          |


6000
59990
1
8.318544387817383 threshold 18.155363 -70.777794
2
-9.811124801635742 threshold 3.4373322 -78.714905
  20 +------------------------------------------------------------------------------------------+
     |                                                                                          |
     |                                                     V_{m} (mV), versus time (ms) ******* |
     |                                                                                          |
     |                     ********* ***** *  *  * *   *                                        |
     |                     ********* ************* ** *********** ******** ** *** * *           |
     |                     ********* ************* ** ******************** *************        |
     |                     ********* ************* ** ******************** *************        |
  10 |                     ********* ************* ** ******************** *************        |
    

12000
119990
3
35.48707437515259 threshold 41.029247 -71.05904
2
30.64351749420166 threshold 40.31053 -69.059135
  40 +------------------------------------------------------------------------------------------+
     |                                                                                          |
     |                                                     V_{m} (mV), versus time (ms) ******* |
     |                                                                                          |
     |                                                    *                                     |
     |                                                    *                                     |
     |                                                    **                                    |
     |                                                    **                                    |
     |                                                    **                                    |
     

...

12000
119990
4
36.06437587738037 threshold 40.56052 -73.24642
1
33.13403940200806 threshold 39.71681 -78.02744
  20 +------------------------------------------------------------------------------------------+
     |                       * ** ******* ****** **** **** **** *** **** *** ** *** ***         |
     |                      ***** ******* ****** **** **** ****}********e***s**i***(*** ******* |
     |                      ***** ******* ****** **** **** **** ******** *** ** *** ****        |
     |                     ****** ******* ****** **** **** **** ******** *** ** *** ****        |
     |                     ****** ******* ****** **** **** **** ******** *** ** *** ****        |
     |                     ****** ******* ****** **** **** **** ******** *** ** *** ****        |
     |                     ****** ******* ****** **** **** **** ******** *** ** *** ****        |
  10 |                     ****** ************** **** **** **** ******** *** ** *** ****        |
     | 

12000
119990
49
9.43541145324707 threshold 15.624237 -72.96519
49
9.43541145324707 threshold 15.624237 -72.96519
  20 +------------------------------------------------------------------------------------------+
     |                    *                                                                     |
     |                    *                                V_{m} (mV), versus time (ms) ******* |
     |                    * ******                                                              |
     |                    *************** * * * *                                               |
     |                    *********************************** **   *                            |
     |                    *********************************************************** *         |
     |                    *************************************************************         |
  10 |                    *************************************************************         |
     

12000
119990
9
6.654960632324219 threshold 13.499341 -80.99605
8
10.187304496765137 threshold 18.62409 -74.15263
  40 +------------------------------------------------------------------------------------------+
     |                                                                                          |
     |                                                     V_{m} (mV), versus time (ms) ******* |
     |                                                                                          |
     |                                                                                          |
     |                                                                                          |
     |                                                                                          |
     |                                                                                          |
     |                                                                                          |
     

12000
119990
57
13.144844055175781 threshold 21.186466 -72.15273
55
7.822220802307129 threshold 18.342855 -78.27743
  40 +------------------------------------------------------------------------------------------+
     |                                                                                          |
     |                                                     V_{m} (mV), versus time (ms) ******* |
     |                                                                                          |
     |                                                                                          |
     |                     *                                                                    |
     |                     *                                                                    |
     |                     *                                                                    |
     |                     *                                         *                          |
  

12000
119990


In [ ]:
bbp = pd.DataFrame([m.everything for m in models_hbp])

In [ ]:
features

In [ ]:
#from neuronunit import bbp
#list_curated_data()


threshes = []

def find_nearest(array,value):
    array = np.asarray(array)
    idx = (np.abs(array-value)).argmin()
    return (array[idx],idx)
def plot_rheobase(data):
    currents = {}
    vms = {}
    rekey3 = {}
    rekey0 = {}
    for k,v in data.items():
        if "Ch3" in k:
            rekey3[k] = v
        if "Ch0" in k:
            rekey0[k] = v
    vms = natsort.humansorted([(k,v) for k,v in rekey3.items()])
    injections = natsort.humansorted([(k,v) for k,v in rekey0.items()])
    threshes = []
    for j,(k,v) in enumerate(vms):
        threshold = spike_functions.threshold_detection(v)
        threshes.append(len(threshold))
    for j,(k,v) in enumerate(vms):
        threshold = spike_functions.threshold_detection(v)
        if len(threshold) == np.min(threshes) and np.max(v.times)>2.8*pq.s:

            target = j
            plot_data(vms[target][1],injections[target][1])
import quantities as pq
    
    
def like_plot_rheobase(data):
    currents = {}
    vms = {}
    rekey3 = {}
    rekey0 = {}
    for k,v in data.items():
        if "Ch3" in k:
            rekey3[k] = v
        if "Ch0" in k:
            rekey0[k] = v
    vms = natsort.humansorted([(k,v) for k,v in rekey3.items()])
    injections = natsort.humansorted([(k,v) for k,v in rekey0.items()])
    comprehendable = [np.max(ind[1]) for ind in injections if ind[1].units==pq.pA]
    print(comprehendable)
    threshes = []
    
    for _,(_,v) in enumerate(vms):
        threshold = spike_functions.threshold_detection(v)
        if np.max(v.times)>2.8*pq.s and len(threshold):            
            threshes.append(len(threshold))
        
    for j,(k,v) in enumerate(vms):
        threshold = spike_functions.threshold_detection(v)
        if len(threshold) == np.min(threshes) and np.max(v.times)>2.8*pq.s:
            rheobase_vm = v
            model = StaticModel(rheobase_vm)
            model.protocol = None
            model.inj = []
            one_five = float(comprehendable[j])*1.5
            three = float(comprehendable[j])*3.0
            one_five = find_nearest(comprehendable,one_five)
            model.rheobase_current = comprehendable[j]
            model.vm15 = vms[one_five[1]]
            three = find_nearest(comprehendable,three)
            model.vm30 = vms[three[1]]
            return model, one_five,three





In [ ]:
from neuronunit.models import StaticModel
def get_15_30(data):
    currents = {}
    vms = {}
    rekey3 = {} # Membrane Potentials
    rekey0 = {} # Current injections
    for k,v in data.items():
        if "Ch3" in k:
            rekey3[k] = v
        if "Ch0" in k:
            rekey0[k] = v
    vms = natsort.humansorted([(k,v) for k,v in rekey3.items()])
    injections = natsort.humansorted([(k,v) for k,v in rekey0.items()])
    inj_list = [i for i in injections]

    threshes = []
    for j,(k,v) in enumerate(vms):
        threshold = spike_functions.threshold_detection(v)
        threshes.append(len(threshold))
    for j,(k,v) in enumerate(vms):
        threshold = spike_functions.threshold_detection(v)
        if len(threshold) == np.min(threshes) and np.max(v.times)>2.8*pq.s:
            target_idx = j
            rheobase = injections[target_idx][1]
            vmrh = (vms[target_idx][1],rheobase)
            model = StaticModel(vms[target_idx][1])
            plot_rheobase(vmrh)
            return (model,inj_list)

    #print(np.max([inj[1] for inj in injections ]))    
#get_15_30(data)    
    #np.closest()

In [ ]:
#injections
import pickle
models = pickle.load(open('models.p','rb'))    

from neuronunit.optimisation.get_three_feature_sets_from_nml_db import three_feature_sets_on_static_models
'''
data_ids = list_curated_data()[0:5]
tuples = []
models = []
results = []
for di in data_ids:
    data = get_curated_data(di)
    plot_rheobase(data)
'''
from neo import AnalogSignal
for m in models:
    print(m)
    #print(AnalogSignal(m.vm30,)
    #print(m[0],m[1])
#    print(dir(m[0]))
#    print(type(m[0]))()
    #features = three_feature_sets_on_static_models(m)
    
#np.shape(models)    


#models[2][-1]

In [ ]:
import pickle

try:
    data = pickle.load(open('models.p','rb'))    
except:
    data_ids = list_curated_data()[0:5]
    tuples = []
    models = []
    for di in data_ids:

        data = get_curated_data(di)
        out = like_plot_rheobase(data)        
        if out is not None:
            models.append([di,out[0]])

    pickle.dump(models,open('models.p','wb'))        


In [ ]:
import pickle

try:
    assert 1==2
    data = pickle.load(open('content.p','rb'))    
except:
    data_ids = list_curated_data()[0:5]
    tuples = []
    models = []
    for di in data_ids:

        data = get_curated_data(di)
        out = like_plot_rheobase(data)        
        if out is not None:
            models.append([di,out[0]])
            features = three_feature_sets_on_static_models(models[-1])

        '''
        currents = {}
        vms = {}
        rekey3 = {} # Membrane Potentials
        rekey0 = {} # Current injections
        for k,v in data.items():
            if "Ch3" in k:
                rekey3[k] = v
            if "Ch0" in k:
                rekey0[k] = v
        vms = natsort.humansorted([(k,v) for k,v in rekey3.items()])
        injections = natsort.humansorted([(k,v) for k,v in rekey0.items()])
        '''
        
    pickle.dump(models,open('models.p','wb'))        
        #out_tuple = get_15_30(data)    
        #tuples.append(out_tuple)

In [ ]:
data = pickle.load(open('content.p','rb'))    
#data


In [ ]:
data.keys

In [ ]:
#plot_rheobase(data)
#print(out)
#out[1].
#efel_evaluation()
#injections[0]#.values()
#type(tuples[0][1][0#])

#plot_rheobase(data)


    #all_sweeps = list_sweeps(did)
    #break
models = pickle.load(open('models.p','rb'))    
#features = three_feature_sets_on_static_models(models[-1])
models[0].vm30[2]

In [ ]:
out[0]

In [ ]:
print(out[2])
one_five = float(out[2][0])*1.5
three = float(out[2][0])*3.0

one_five = find_nearest(out[2],one_five)
three = find_nearest(out[2],three)

#print([i[2] for i in out)
#len(out)
three

In [ ]:
vms[1]


In [ ]:
print(type(data))

In [ ]:
data[list(data.keys())[1]].units

In [ ]:
data[list(data.keys())[0]]

In [ ]:
data[list(data.keys())[1]]

In [ ]:
rekey3;

In [ ]:
!pwd

In [ ]:
help(natsort)

In [ ]:
plot_rheobase(data)
